This notebook creates a new directory, *cropped_images*, of images by cropping the bounding boxes in each original image.

In [1]:
import os
import shutil

img_path = '../images'
img_files = [os.path.join(img_path, f) for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))]
ann_path = '../annotations'
ann_files = [os.path.join(ann_path, f) for f in os.listdir(ann_path) if os.path.isfile(os.path.join(ann_path, f))]

cropped_img_path = '../cropped_images'
if os.path.isdir(cropped_img_path):
    shutil.rmtree(cropped_img_path)
os.mkdir(cropped_img_path)

In [2]:
import xml.etree.ElementTree as ET
from skimage import io

margin = 10
for i, file in enumerate(ann_files):
        # Open image
        img = io.imread(img_files[i])
    
        # Read XML
        in_file = open(file)
        tree=ET.parse(in_file)
        root = tree.getroot() 
        imsize = root.find('size')
        w = int(imsize.find('width').text)
        h = int(imsize.find('height').text)
        
        j = 0
        for obj in root.iter('object'):
            name = obj.find('name').text

            xmlbox = obj.find('bndbox')
            xn = int(float(xmlbox.find('xmin').text))
            xx = int(float(xmlbox.find('xmax').text))
            yn = int(float(xmlbox.find('ymin').text))
            yx = int(float(xmlbox.find('ymax').text))
            
            xn = max(0, xn - margin)
            xx = min(w, xx + margin)
            yn = max(0, yn - margin)
            yx = min(h, yx + margin)
            
            cropped_img = img[yn:yx,xn:xx]
            cropped_img_file = os.path.join(cropped_img_path, img_files[i][len(img_path) + 1:])
            if (j > 0):
                cropped_img_file = cropped_img_file[:-4] + '-%d'.format(j) + cropped_img_file[-4:]
            io.imsave(cropped_img_file, cropped_img)
            j = j + 1